In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# In Kaggle notebooks, use this safe install approach
!pip install torch==2.3.0 torch-geometric==2.5.3 codecarbon==2.3.5 -q
!pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.3.0+cu121.html -q

import torch, torch_geometric, codecarbon
print("PyTorch:", torch.__version__)
print("PyG:", torch_geometric.__version__)
print("CodeCarbon:", codecarbon.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("✅ Device in use:", device)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 608.4 kB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.6/174.6 kB 201.4 kB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 88.2 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 14.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 31.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# PyG dataset auto-loader (downloads only first time)
from torch_geometric.datasets import Planetoid, Amazon, TUDataset, WikiCS

# Node classification datasets
cora = Planetoid(root="./data/Cora", name="Cora")
citeseer = Planetoid(root="./data/CiteSeer", name="CiteSeer")
pubmed = Planetoid(root="./data/PubMed", name="PubMed")

# Large node-level datasets
amazon_photo = Amazon(root="./data/AmazonPhoto", name="Photo")
amazon_computers = Amazon(root="./data/AmazonComputers", name="Computers")

# Protein and molecule graph datasets
proteins = TUDataset(root="./data/PROTEINS", name="PROTEINS")
enzymes = TUDataset(root="./data/ENZYMES", name="ENZYMES")
nci1 = TUDataset(root="./data/NCI1", name="NCI1")
mutag = TUDataset(root="./data/MUTAG", name="MUTAG")

# Wiki dataset (e.g., link prediction / node classification)
wikics = WikiCS(root="./data/WikiCS")


Processing...
Done!
Processing...
Done!
Processing...
Done!
Processing...
Done!
Processing...
Done!
Processing...
Done!
Processing...
Done!
Processing...
Done!
Processing...
Done!
/tmp/ipykernel_37/1179130855.py:20: UserWarning: The WikiCS dataset now returns an undirected graph by default. Please explicitly specify 'is_undirected=False' to restore the old behavior.
  wikics = WikiCS(root="./data/WikiCS")
Processing...
Done!


In [5]:
!pip install torch torch-geometric codecarbon --quiet
from torch_geometric.datasets import Planetoid, Amazon, TUDataset, WikiCS
from torch_geometric.nn import GCNConv, GATConv, SAGEConv, global_mean_pool
from torch_geometric.loader import DataLoader
from codecarbon import EmissionsTracker
import torch
import torch.nn.functional as F
import pandas as pd, time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# -------- Datasets (Node + Graph level) --------
datasets = {
    "Cora": lambda: Planetoid(root="./data/Cora", name="Cora"),
    "CiteSeer": lambda: Planetoid(root="./data/CiteSeer", name="CiteSeer"),
    "PubMed": lambda: Planetoid(root="./data/PubMed", name="PubMed"),
    "Amazon-Photo": lambda: Amazon(root="./data/AmazonPhoto", name="Photo"),
    "Amazon-Computers": lambda: Amazon(root="./data/AmazonComputers", name="Computers"),
    "ENZYMES": lambda: TUDataset(root="./data/ENZYMES", name="ENZYMES"),
    "PROTEINS": lambda: TUDataset(root="./data/PROTEINS", name="PROTEINS"),
    "MUTAG": lambda: TUDataset(root="./data/MUTAG", name="MUTAG"),
    "NCI1": lambda: TUDataset(root="./data/NCI1", name="NCI1"),
    "WikiCS": lambda: WikiCS(root="./data/WikiCS")
}

# ======== Model Architectures ========
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden, out_channels, graph_level=False):
        super().__init__()
        self.graph_level = graph_level
        self.conv1 = GCNConv(in_channels, hidden)
        self.conv2 = GCNConv(hidden, out_channels)
    def forward(self, x, edge_index, batch=None):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        if self.graph_level:
            x = global_mean_pool(x, batch)
        return x

class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden, out_channels, graph_level=False):
        super().__init__()
        self.graph_level = graph_level
        self.conv1 = GATConv(in_channels, hidden, heads=8)
        self.conv2 = GATConv(hidden*8, out_channels, heads=1, concat=False)
    def forward(self, x, edge_index, batch=None):
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        if self.graph_level:
            x = global_mean_pool(x, batch)
        return x

class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden, out_channels, graph_level=False):
        super().__init__()
        self.graph_level = graph_level
        self.conv1 = SAGEConv(in_channels, hidden)
        self.conv2 = SAGEConv(hidden, out_channels)
    def forward(self, x, edge_index, batch=None):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        if self.graph_level:
            x = global_mean_pool(x, batch)
        return x

# ======== Training Utilities ========
def train_node_model(model, data):
    model.to(device); data = data.to(device)
    opt = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    for epoch in range(200):
        model.train(); opt.zero_grad()
        out = model(data.x, data.edge_index)
        loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
        loss.backward(); opt.step()
    model.eval()
    pred = out.argmax(dim=1)
    acc = (pred[data.test_mask] == data.y[data.test_mask]).sum() / int(data.test_mask.sum())
    return acc.item() * 100

def train_graph_model(model, dataset):
    loader = DataLoader(dataset, batch_size=32, shuffle=True)
    model.to(device)
    opt = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    criterion = torch.nn.CrossEntropyLoss()

    # simple split 80% train, 20% test
    train_size = int(0.8 * len(dataset))
    train_ds, test_ds = torch.utils.data.random_split(dataset, [train_size, len(dataset)-train_size])
    train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)

    for epoch in range(50):
        model.train()
        for batch in train_loader:
            batch = batch.to(device)
            out = model(batch.x, batch.edge_index, batch.batch)
            loss = criterion(out, batch.y)
            opt.zero_grad(); loss.backward(); opt.step()

    model.eval(); correct, total = 0, 0
    with torch.no_grad():
        for batch in test_loader:
            batch = batch.to(device)
            out = model(batch.x, batch.edge_index, batch.batch)
            pred = out.argmax(dim=1)
            correct += (pred == batch.y).sum().item()
            total += batch.num_graphs
    return 100.0 * correct / total

# ======== Run Experiment Across All Datasets ========
results = []
models = {"GCN": GCN, "GAT": GAT, "GraphSAGE": GraphSAGE}

for dname, loader in datasets.items():
    try:
        dataset = loader()
        data = dataset[0]
        if hasattr(data, "train_mask"):   # Node-level
            in_ch, out_ch = data.num_features, int(data.y.max().item()) + 1
            for mname, M in models.items():
                tracker = EmissionsTracker(project_name=f"{mname}_{dname}")
                tracker.start(); start = time.time()

                model = M(in_ch, 64, out_ch, graph_level=False)
                acc = train_node_model(model, data)
                carbon = tracker.stop(); duration = round(time.time() - start, 2)

                print(f"{dname:<15} | {mname:<10} | Node | Acc: {acc:6.2f}% | CO2: {carbon:.6f} kg | Time: {duration}s")
                results.append([dname, mname, acc, carbon])
        else:                            # Graph-level dataset
            in_ch, out_ch = dataset.num_features, dataset.num_classes
            for mname, M in models.items():
                tracker = EmissionsTracker(project_name=f"{mname}_{dname}")
                tracker.start(); start = time.time()

                model = M(in_ch, 64, out_ch, graph_level=True)
                acc = train_graph_model(model, dataset)
                carbon = tracker.stop(); duration = round(time.time() - start, 2)

                print(f"{dname:<15} | {mname:<10} | Graph | Acc: {acc:6.2f}% | CO2: {carbon:.6f} kg | Time: {duration}s")
                results.append([dname, mname, acc, carbon])
    except Exception as e:
        print(f"{dname:<15} | Error: {e}")

df = pd.DataFrame(results, columns=["Dataset", "Model", "Accuracy(%)", "CO2(kg)"])
df.to_csv("/kaggle/working/gnn_all_results.csv", index=False)
print("\nResults saved to /kaggle/working/gnn_all_results.csv")
df.head(10)


[codecarbon INFO @ 10:54:09] Energy consumed for RAM : 0.026569 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 10:54:09] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 10:54:09] Energy consumed for All CPU : 0.056463 kWh
[codecarbon INFO @ 10:54:10] Energy consumed for all GPUs : 0.057164 kWh. Total GPU Power : 43.27316698890296 W
[codecarbon INFO @ 10:54:10] 0.140196 kWh of electricity used since the beginning.
[codecarbon WARNING @ 10:54:12] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 10:54:13] [setup] RAM Tracking...
[codecarbon INFO @ 10:54:13] [setup] CPU Tracking...
[codecarbon WARNING @ 10:54:14] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 10:54:14] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem t

Cora            | GCN        | Node | Acc:  78.90% | CO2: 0.000009 kg | Time: 0.65s


[codecarbon INFO @ 10:54:18] Energy consumed for RAM : 0.026569 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 10:54:18] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 10:54:18] Energy consumed for All CPU : 0.056463 kWh
[codecarbon INFO @ 10:54:18] Energy consumed for all GPUs : 0.057180 kWh. Total GPU Power : 43.82731032769674 W
[codecarbon INFO @ 10:54:18] 0.140212 kWh of electricity used since the beginning.
[codecarbon WARNING @ 10:54:19] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 10:54:19] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 10:54:19] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon WARNING @ 10:54:19] No CPU tracking mode found. Falling back on CPU constant mode

Cora            | GAT        | Node | Acc:  72.50% | CO2: 0.000028 kg | Time: 1.71s


[codecarbon INFO @ 10:54:24] Energy consumed for RAM : 0.026652 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 10:54:24] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 10:54:24] Energy consumed for All CPU : 0.056640 kWh
[codecarbon INFO @ 10:54:25] Energy consumed for all GPUs : 0.057361 kWh. Total GPU Power : 47.48102494867697 W
[codecarbon INFO @ 10:54:25] 0.140654 kWh of electricity used since the beginning.
[codecarbon INFO @ 10:54:25] 0.013380 g.CO2eq/s mean an estimation of 421.9663751296945 kg.CO2eq/year
[codecarbon WARNING @ 10:54:25] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 10:54:25] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 10:54:25] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @

...

NCI1            | GraphSAGE  | Graph | Acc:  62.17% | CO2: 0.000329 kg | Time: 24.75s


[codecarbon INFO @ 10:58:19] Energy consumed for RAM : 0.028068 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 10:58:19] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 10:58:19] Energy consumed for All CPU : 0.059649 kWh
[codecarbon INFO @ 10:58:19] Energy consumed for all GPUs : 0.060601 kWh. Total GPU Power : 43.468701533368026 W
[codecarbon INFO @ 10:58:19] 0.148318 kWh of electricity used since the beginning.
[codecarbon WARNING @ 10:58:20] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon WARNING @ 10:58:20] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 10:58:20] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon WARNING @ 10:58:20] No CPU tracking mode found. Falling back on CPU constant mod

WikiCS          | Error: The shape of the mask [11701, 20] at index 1 does not match the shape of the indexed tensor [11701, 10] at index 1

Results saved to /kaggle/working/gnn_all_results.csv


,Dataset,Model,Accuracy(%),CO2(kg)
0,Cora,GCN,78.900003,0.000009
1,Cora,GAT,72.500002,0.000028
2,Cora,GraphSAGE,78.600001,0.000013
3,CiteSeer,GCN,65.400004,0.000012
4,CiteSeer,GAT,60.300004,0.000041
5,CiteSeer,GraphSAGE,65.800005,0.000022
6,PubMed,GCN,76.200002,0.000015
7,PubMed,GAT,76.100004,0.000108
8,PubMed,GraphSAGE,76.100004,0.000027
9,ENZYMES,GCN,21.666667,0.000052


[codecarbon INFO @ 10:58:24] Energy consumed for RAM : 0.028068 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 10:58:24] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 10:58:24] Energy consumed for All CPU : 0.059649 kWh
[codecarbon INFO @ 10:58:24] Energy consumed for all GPUs : 0.060609 kWh. Total GPU Power : 43.400594285042104 W
[codecarbon INFO @ 10:58:24] 0.148325 kWh of electricity used since the beginning.
[codecarbon INFO @ 10:58:24] Energy consumed for RAM : 0.027985 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 10:58:25] Delta energy consumed for CPU with constant : 0.000177 kWh, power : 42.5 W
[codecarbon INFO @ 10:58:25] Energy consumed for All CPU : 0.059472 kWh
[codecarbon INFO @ 10:58:25] Energy consumed for all GPUs : 0.060455 kWh. Total GPU Power : 43.16572490359413 W
[codecarbon INFO @ 10:58:25] 0.147911 kWh of electricity used since the beginning.
[codecarbon INFO @ 10:58:25] 0.013454 g.CO2eq/s mean an estimation of 424.2897791

In [ ]:
class GAT(torch.nn.Module):
    def __init__(self, in_feats, hidden, out_feats, heads=8):
        super().__init__()
        self.conv1 = GATConv(in_feats, hidden, heads=heads)
        self.conv2 = GATConv(hidden*heads, out_feats, heads=1, concat=False)
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, 0.6, training=self.training)
        return self.conv2(x, edge_index)

results = []
for name, loader in datasets.items():
    try:
        data_obj = loader()
        data = data_obj[0].to(device)
        model = GAT(data.num_features, 8, data.y.max().item()+1)
        tracker = EmissionsTracker(project_name=f'GAT_{name}')
        tracker.start(); start = time.time()
        acc = train_and_eval(model, data)
        carbon = tracker.stop()
        results.append([name, acc, carbon])
        print(f"{name:<18} | Accuracy: {acc:6.2f}% | CO2: {carbon:.6f} kg")
    except Exception as e:
        print(f"{name:<18} | Error: {e}")
pd.DataFrame(results, columns=['Dataset','Accuracy(%)','CO2(kg)'])


In [ ]:
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_feats, hidden, out_feats):
        super().__init__()
        self.conv1 = SAGEConv(in_feats, hidden)
        self.conv2 = SAGEConv(hidden, out_feats)
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, 0.5, training=self.training)
        return self.conv2(x, edge_index)

results = []
for name, loader in datasets.items():
    try:
        data_obj = loader()
        data = data_obj[0].to(device)
        model = GraphSAGE(data.num_features, 64, data.y.max().item()+1)
        tracker = EmissionsTracker(project_name=f'SAGE_{name}')
        tracker.start(); start = time.time()
        acc = train_and_eval(model, data)
        carbon = tracker.stop()
        results.append([name, acc, carbon])
        print(f"{name:<18} | Accuracy: {acc:6.2f}% | CO2: {carbon:.6f} kg")
    except Exception as e:
        print(f"{name:<18} | Error: {e}")
pd.DataFrame(results, columns=['Dataset','Accuracy(%)','CO2(kg)'])


In [16]:
!pip install torch torch-geometric codecarbon --quiet
from torch_geometric.datasets import Planetoid, Amazon, TUDataset, WikiCS
from torch_geometric.nn import GCN2Conv, GINConv, GENConv, DeepGCNLayer, global_mean_pool, GATConv, SAGEConv
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import RandomNodeSplit
from codecarbon import EmissionsTracker
import torch, torch.nn.functional as F, pandas as pd, time, os

# Suppress CodeCarbon's noisy output for cleaner benchmark results
import logging
logging.getLogger('codecarbon').setLevel(logging.CRITICAL)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Active device: {device}")

# ---------------------------------------------------
# Local paths in Kaggle (/kaggle/working/data)
# ---------------------------------------------------
data_root = "/kaggle/working/data"

datasets = {
    "Cora": Planetoid(root=os.path.join(data_root, "Cora"), name="Cora"),
    "CiteSeer": Planetoid(root=os.path.join(data_root, "CiteSeer"), name="CiteSeer"),
    "PubMed": Planetoid(root=os.path.join(data_root, "PubMed"), name="PubMed"),
    "Amazon-Photo": Amazon(root=os.path.join(data_root, "AmazonPhoto"), name="Photo", transform=RandomNodeSplit(split='train_rest', num_train_per_class=20, num_val=500, num_test=1000)),
    "Amazon-Computers": Amazon(root=os.path.join(data_root, "AmazonComputers"), name="Computers", transform=RandomNodeSplit(split='train_rest', num_train_per_class=20, num_val=500, num_test=1000)),
    "WikiCS": WikiCS(root=os.path.join(data_root, "WikiCS")),
    "ENZYMES": TUDataset(root=os.path.join(data_root, "ENZYMES"), name="ENZYMES"),
    "PROTEINS": TUDataset(root=os.path.join(data_root, "PROTEINS"), name="PROTEINS"),
    "MUTAG": TUDataset(root=os.path.join(data_root, "MUTAG"), name="MUTAG"),
    "NCI1": TUDataset(root=os.path.join(data_root, "NCI1"), name="NCI1"),
}

# ---------------------------------------------------
# Model Definitions
# ---------------------------------------------------
class GCNII(torch.nn.Module):
    def __init__(self, in_feats, hidden, out_feats, num_layers=6):
        super().__init__()
        self.lin1 = torch.nn.Linear(in_feats, hidden)
        self.convs = torch.nn.ModuleList([GCN2Conv(hidden, 0.1, 0.5, layer=i+1) for i in range(num_layers)])
        self.lin2 = torch.nn.Linear(hidden, out_feats)
    def forward(self, x, edge_index):
        x = F.dropout(self.lin1(x), 0.6, training=self.training)
        x0 = x.clone()
        for conv in self.convs:
            x = F.dropout(F.relu(conv(x, x0, edge_index)), 0.6, training=self.training)
        return self.lin2(x)

class GAT(torch.nn.Module):
    def __init__(self, in_feats, hidden, out_feats, heads=8):
        super().__init__()
        self.conv1 = GATConv(in_feats, hidden, heads=heads, dropout=0.6)
        self.conv2 = GATConv(hidden * heads, out_feats, heads=1, concat=False, dropout=0.6)
    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return x

class GraphSAGE(torch.nn.Module):
    def __init__(self, in_feats, hidden, out_feats):
        super().__init__()
        self.conv1 = SAGEConv(in_feats, hidden)
        self.conv2 = SAGEConv(hidden, out_feats)
    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        return x

class GIN(torch.nn.Module):
    def __init__(self, in_feats, hidden, out_feats):
        super().__init__()
        nn1 = torch.nn.Sequential(torch.nn.Linear(in_feats, hidden), torch.nn.ReLU(), torch.nn.Linear(hidden, hidden))
        self.conv1 = GINConv(nn1)
        nn2 = torch.nn.Sequential(torch.nn.Linear(hidden, hidden), torch.nn.ReLU(), torch.nn.Linear(hidden, hidden))
        self.conv2 = GINConv(nn2)
        self.lin = torch.nn.Linear(hidden, out_feats)
    def forward(self, x, edge_index, batch):
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = global_mean_pool(x, batch)
        return self.lin(x)

class DeepGCN(torch.nn.Module):
    # --- 💡 FIX: Added an initial linear layer to project input features to the hidden dimension ---
    def __init__(self, in_feats, hidden, out_feats, layers=4):
        super().__init__()
        self.lin1 = torch.nn.Linear(in_feats, hidden)
        self.layers = torch.nn.ModuleList()
        for _ in range(layers):
            self.layers.append(DeepGCNLayer(
                GENConv(hidden, hidden),      # All layers now operate on the hidden dimension
                norm=torch.nn.LayerNorm(hidden),
                act=torch.nn.ReLU(),
                block='res+'
            ))
        self.lin2 = torch.nn.Linear(hidden, out_feats)

    def forward(self, x, edge_index, batch):
        # 1. Apply initial embedding
        x = self.lin1(x)
        # 2. Pass through deep GCN layers
        for layer in self.layers:
            x = layer(x, edge_index)
        # 3. Pool and apply final classifier
        x = global_mean_pool(x, batch)
        x = self.lin2(x)
        return x

# ---------------------------------------------------
# Training & Evaluation Utilities
# ---------------------------------------------------
def train_node_model(model, data, is_wikics=False):
    data, model = data.to(device), model.to(device)
    if isinstance(model, GAT):
        opt = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)
    else:
        opt = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    train_mask = data.train_mask[:, 0] if is_wikics else data.train_mask
    for epoch in range(1, 101):
        model.train(); opt.zero_grad()
        out = model(data.x, data.edge_index)
        loss = F.cross_entropy(out[train_mask], data.y[train_mask])
        loss.backward(); opt.step()
    model.eval()
    with torch.no_grad():
        preds = model(data.x, data.edge_index).argmax(dim=1)
        test_mask = data.val_mask[:, 0] if is_wikics else data.test_mask
        acc = (preds[test_mask] == data.y[test_mask]).float().mean().item()*100
    return round(acc, 2)

def train_graph_model(model, dataset):
    n = len(dataset)
    tr_len = max(1, int(0.8 * n))
    train_ds, test_ds = torch.utils.data.random_split(dataset, [tr_len, n - tr_len])
    train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_ds, batch_size=64)
    opt = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
    loss_fn = torch.nn.CrossEntropyLoss()
    model = model.to(device)
    for _ in range(40):
        model.train()
        for batch in train_loader:
            batch = batch.to(device)
            out = model(batch.x.float(), batch.edge_index, batch.batch)
            loss = loss_fn(out, batch.y)
            opt.zero_grad(); loss.backward(); opt.step()
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for batch in test_loader:
            batch = batch.to(device)
            preds = model(batch.x.float(), batch.edge_index, batch.batch).argmax(dim=1)
            correct += (preds == batch.y).sum().item()
            total += batch.num_graphs
    return round(100 * correct / total, 2) if total else 0.0

# ---------------------------------------------------
# Run Benchmark
# ---------------------------------------------------
results = []
node_models = {"GCNII": GCNII, "GAT": GAT, "GraphSAGE": GraphSAGE}
graph_models = {"GIN": GIN, "DeepGCN": DeepGCN}

for dname, dataset in datasets.items():
    try:
        data = dataset[0]
        is_node_task = hasattr(data, "train_mask")
        in_ch, out_ch = dataset.num_features, dataset.num_classes

        models_to_run = node_models if is_node_task else graph_models

        for mname, Model in models_to_run.items():
            tracker = EmissionsTracker(project_name=f"{mname}_{dname}", output_dir='/tmp/', log_level='error')
            tracker.start(); start = time.time()

            model = Model(in_ch if in_ch > 0 else 1, 64, out_ch)

            if is_node_task and hasattr(data, 'x') and data.x is not None:
                data.x = data.x.float()
            elif is_node_task:
                data.x = torch.ones((data.num_nodes, 1), dtype=torch.float)

            if is_node_task:
                acc = train_node_model(model, data, is_wikics=(dname == 'WikiCS'))
            else:
                acc = train_graph_model(model, dataset)

            carbon = tracker.stop(); duration = round(time.time() - start, 2)

            print(f"{dname:<17} | {mname:<10} | Acc: {acc:6.2f}% | CO2: {carbon:.6f} kg | Time: {duration:6.2f}s")
            results.append([dname, mname, acc, carbon, duration])

    except Exception as e:
        import traceback
        print(f"-> Critical Error on {dname}: {e}")
        traceback.print_exc()

df = pd.DataFrame(results, columns=["Dataset", "Model", "Accuracy(%)", "CO2 (kg)", "Time(s)"])
df.to_csv("/kaggle/working/complex_gnn_final_results.csv", index=False)
print("\n✅ All results saved to /kaggle/working/complex_gnn_final_results.csv")
print("\n--- Final Results ---")
print(df.to_string())

✅ Active device: cuda


/tmp/ipykernel_37/59533810.py:27: UserWarning: The WikiCS dataset now returns an undirected graph by default. Please explicitly specify 'is_undirected=False' to restore the old behavior.
  "WikiCS": WikiCS(root=os.path.join(data_root, "WikiCS")),


Cora              | GCNII      | Acc:  79.50% | CO2: 0.000013 kg | Time:   0.95s
Cora              | GAT        | Acc:  79.90% | CO2: 0.000014 kg | Time:   0.86s
Cora              | GraphSAGE  | Acc:  79.70% | CO2: 0.000006 kg | Time:   0.43s
CiteSeer          | GCNII      | Acc:  68.10% | CO2: 0.000014 kg | Time:   0.97s
CiteSeer          | GAT        | Acc:  64.40% | CO2: 0.000022 kg | Time:   1.36s
CiteSeer          | GraphSAGE  | Acc:  69.70% | CO2: 0.000012 kg | Time:   0.71s
PubMed            | GCNII      | Acc:  75.90% | CO2: 0.000034 kg | Time:   1.97s
PubMed            | GAT        | Acc:  77.30% | CO2: 0.000056 kg | Time:   3.26s
PubMed            | GraphSAGE  | Acc:  77.10% | CO2: 0.000014 kg | Time:   0.84s
Amazon-Photo      | GCNII      | Acc:  93.70% | CO2: 0.000044 kg | Time:   2.56s
Amazon-Photo      | GAT        | Acc:  93.10% | CO2: 0.000083 kg | Time:   4.84s
Amazon-Photo      | GraphSAGE  | Acc:  93.00% | CO2: 0.000035 kg | Time:   2.04s
Amazon-Computers  | GCNII   